# 微调和部署大语言模型ChatGLM2-6b

大型语言模型（Large Language Model, LLM）是基于大量数据进行预训练的超大型语言模型，通过在大量文本数据上进行训练，这些模型能够学习语言的复杂结构和细微含义，并在从文本生成、语言理解等任务上展现出卓越的性能。

PAI 提供了许多热门的开源大语言模型，支持用户开箱即用的方式使用这些模型进行部署或是使用私有数据进行微调训练。当前示例，我们将以ChatGLM2-6b为例，介绍如何使用PAI提供的大语言模型进行微调和部署。



## 费用说明

本示例将会使用以下云产品，并产生相应的费用账单：

- PAI-DLC：运行训练任务，详细计费说明请参考[PAI-DLC计费说明](https://help.aliyun.com/zh/pai/product-overview/billing-of-dlc)
- PAI-EAS：部署推理服务，详细计费说明请参考[PAI-EAS计费说明](https://help.aliyun.com/zh/pai/product-overview/billing-of-eas)
- OSS：存储训练任务输出的模型、训练代码、TensorBoard日志等，详细计费说明请参考[OSS计费概述](https://help.aliyun.com/zh/oss/product-overview/billing-overview)


> 通过参与云产品免费试用，使用**指定资源机型**提交训练作业或是部署推理服务，可以免费试用PAI产品，具体请参考[PAI免费试用](https://help.aliyun.com/zh/pai/product-overview/free-quota-for-new-users)。




## 准备工作

通过以下命令安装PAI Python SDK。

In [ ]:
!python -m pip install --upgrade pai



SDK 需要配置访问阿里云服务需要的 AccessKey，以及当前使用的工作空间和OSS Bucket。在 PAI Python SDK 安装之后，通过在 **命令行终端** 中执行以下命令，按照引导配置密钥，工作空间等信息。


```shell

# 以下命令，请在 命令行终端 中执行.

python -m pai.toolkit.config

```

In [ ]:
from pai.session import get_default_session, setup_default_session

# 用户通过以下的方式配置SDK的session.
sess = get_default_session()
if not sess:
    print("config session")
    sess = setup_default_session(
        region_id="<REGION_ID>",  # 例如：cn-beijing
        workspace_id="<WORKSPACE_ID>",  # 例如：12345
        oss_bucket_name="<OSS_BUCKET_NAME>",
    )
# 将当前的配置持久化到 ~/.pai/config.json，SDK默认从对应的路径读取配置初始化默认session。
sess.save_config()

print(sess.workspace_name)


我们可以通过以下代码验证当前的配置。

In [ ]:
import pai
from pai.session import get_default_session

print(pai.__version__)
sess = get_default_session()
print(sess.workspace_id)
print(sess.workspace_name)

## PAI支持的LLM模型

通过`RegisteredModel.list`方法，使用`task="large-language-model"`，`model_provider="pai"`，可以过滤获得PAI支持的所有的大语言模型。

In [ ]:
from pai.model import RegisteredModel
from IPython.display import display, HTML


def table_display(data):
    content = "".join(
        "<tr>{}</tr>".format(
            "".join(
                '<td style="text-align:left;">{}</td>'.format(column) for column in row
            )
        )
        for row in data
    )
    display(HTML(f"<table>{content}</table>"))


data = [
    [
        "ModelName",
        "ModelVersion",
        "ModelProvider",
        "Task",
        "Labels",
        "SupportTraining",
        "SupportDeploy",
    ]
]
for m in RegisteredModel.list(model_provider="pai", task="large-language-model"):
    data.append(
        [
            m.model_name,
            m.model_version,
            m.model_provider,
            m.task,
            ", ".join(["{}={}".format(k, v) for k, v in m.version_labels.items()]),
            bool(m.training_spec),
            bool(m.inference_spec),
        ]
    )


table_display(data)

ModelName,ModelVersion,ModelProvider,Task,Labels,SupportTraining,SupportDeploy
baichuan2-7b-chat,0.2.1,pai,large-language-model,"disabled=True, framework=PyTorch, language=Multilingual, model_size=7000000000, source=ModelScope",True,True
baichuan2-7b-base,0.2.1,pai,large-language-model,"disabled=True, framework=PyTorch, language=Multilingual, model_size=7000000000, source=ModelScope",True,True
qwen-7b-chat,0.2.1,pai,large-language-model,"disabled=True, framework=PyTorch, language=Multilingual, model_size=7000000000, source=ModelScope",True,True
qwen-7b,0.2.1,pai,large-language-model,"disabled=True, framework=PyTorch, language=Multilingual, model_size=7000000000, source=ModelScope",True,True
llama-2-7b-chat,0.2.1,pai,large-language-model,"framework=PyTorch, language=Multi-lingual, model_size=7000000000, source=ModelScope",True,True
llama-2-7b,0.2.1,pai,large-language-model,"framework=PyTorch, language=Multi-lingual, model_size=7000000000, source=ModelScope",True,True
falcon-7b-instruct,0.2.1,pai,large-language-model,"framework=PyTorch, language=Multi-lingual, model_size=7000000000, source=ModelScope",True,True
falcon-7b,0.2.1,pai,large-language-model,"framework=PyTorch, language=Multi-lingual, model_size=7000000000, source=ModelScope",True,True
dolly-v2-7b,0.2.1,pai,large-language-model,"framework=PyTorch, language=Multi-lingual, model_size=7000000000, source=ModelScope",True,True
dolly-v2-3b,0.2.1,pai,large-language-model,"framework=PyTorch, language=Multi-lingual, model_size=3000000000, source=ModelScope",True,True


## 部署ChatGLM模型


PAI提供的这些大语言模型，支持用户直接部署到PAI上，创建一个在线服务。用户可以通过`RegisteredModel.deploy`方法，将模型部署到PAI上，创建一个专属的大语言模型在线服务。

以下我们将以ChatGLM2-6b为示例，介绍如何部署一个大语言模型在线服务。

In [ ]:
from pai.model import RegisteredModel

# 获取PAI提供的ChatGLM2-6b模型
m = RegisteredModel(model_name="chatglm2-6b", model_provider="pai")


# 查看模型的部署配置
print(m.inference_spec)

In [ ]:
from pai.common.utils import random_str

# 部署模型服务
predictor = m.deploy(
    "chatglm2_6b_{}".format(random_str(8)),  # 推理服务名称
    #  instance_type="ecs.gn6v-c8g1.2xlarge",   # 配置使用的机器实例规格
    #  instance_count=1,                        # 配置机器实例个数
)

部署模型服务返回`Predictor`对象，可以用于调用在线服务。

In [ ]:
res = predictor.predict(
    data={
        "prompt": "How to install PyTorch?",
        "system_prompt": "Act like you are programmer with 5+ years of experience.",
        "temperature": 0.8,
    }
)
print(res)

{'response': "To install PyTorch, you can use pip, which is the package installer for Python. Here are the steps to install PyTorch using pip:\n\n1. Open a terminal or command prompt.\n2. Enter the following command to install PyTorch:\n```\npip install torch torchvision\n```\n1. Wait for the installation to complete.\n\nThat's it! You should now have PyTorch installed.",
 'history': [['How to install PyTorch?',
   "To install PyTorch, you can use pip, which is the package installer for Python. Here are the steps to install PyTorch using pip:\n\n1. Open a terminal or command prompt.\n2. Enter the following command to install PyTorch:\n```\npip install torch torchvision\n```\n1. Wait for the installation to complete.\n\nThat's it! You should now have PyTorch installed."]],
 'usage': {'usage': None, 'finish_reason': 'stop'}}

在测试完成之后，需要删除部署的模型服务，避免产生额外的费用。

In [ ]:
predictor.delete_service()

## 微调ChatGLM2-6b模型


PAI提供的这些大语言模型，支持用户使用私有数据进行微调训练。用户可以通过`RegisteredModel.get_estimator`方法，获取相应的模型微调算法，然后使用`Estimator.fit`方法，使用私有数据进行微调训练。

In [ ]:
from pai.model import RegisteredModel

# 获取PAI提供的ChatGLM2-6b模型
m = RegisteredModel("chatglm2-6b", model_provider="pai")

# 获取模型的微调算法
est = m.get_estimator()

查看模型的微调算法支持的超参配置

In [ ]:
hp_defs = [["HyperParameter Name", "Type", "DefaultValue", "Required", "Description"]]
for ch in est.hyperparameter_definitions:
    hp_defs.append(
        [
            ch["Name"],
            ch["Type"],
            est.hyperparameters.get(ch["Name"], ch["DefaultValue"]),
            ch["Required"],
            ch["Description"],
        ]
    )

table_display(hp_defs)


通过`est.hyperparameters`和`est.set_hyperparameters`，用户可以查看和设置微调算法支持的超参配置。

In [ ]:
# 查看当前的算法超参配置
print(est.hyperparameters)

# 设置算法的batch_size
est.set_hyperparameters(per_device_train_batch_size=8)

# 查看算法超参配置
print(est.hyperparameters)

{'model': 'chatglm2', 'learning_rate': '1e-5', 'num_train_epochs': '5', 'per_device_train_batch_size': '16', 'max_seq_len': '512', 'lora_dim': '64', 'gradient_accumulation_steps': 1}
{'model': 'chatglm2', 'learning_rate': '1e-5', 'num_train_epochs': '5', 'per_device_train_batch_size': 8, 'max_seq_len': '512', 'lora_dim': '64', 'gradient_accumulation_steps': 1}


PAI提供微调的实例数据集，用户可以通过`model.get_estimator_inputs()`方法获取PAI微调算法支持的输入，用户可以基于这些输入，测试算法，查看数据然后构建自己的训练数据集。

In [ ]:
from pprint import pprint

# 查看模型自动的默认训练输入信息
training_inputs = m.get_estimator_inputs()

pprint(training_inputs)

{'model': 'oss://pai-quickstart-cn-hangzhou.oss-cn-hangzhou-internal.aliyuncs.com/huggingface/models/chatglm2-6b/main/',
 'train': 'oss://pai-quickstart-cn-hangzhou.oss-cn-hangzhou-internal.aliyuncs.com/huggingface/datasets/llm_instruct/ch_poetry_train.json',
 'validation': 'oss://pai-quickstart-cn-hangzhou.oss-cn-hangzhou-internal.aliyuncs.com/huggingface/datasets/llm_instruct/ch_poetry_test.json'}


模型提供多个算法默认输入，包括以下：

- `model`: 训练算法使用的预训练模型，当前示例中是`ChatGLM2-6b`模型的OSS路径。
- `train`: 算法使用的训练数据集
- `validation`: 算法使用的验证数据集


这些数据集或是模型由PAI提供在公共读的OSS Bucket上，可以被挂载到训练作业中，供训练算法代码使用，用户也可以下载相应的数据集到本地，查看数据集的格式。


In [ ]:
from pai.common.oss_utils import download

from pai.session import get_default_session

region_id = get_default_session().region_id


# 下载训练数据集到本地
local_path = download(
    f"oss://pai-quickstart-{region_id}/huggingface/datasets/llm_instruct/ch_poetry_train.json",
    "./train_data/",
)

大语言模型的微调训练任务，支持通过`JSON`的形式提供数据集，每一行是一个`Dict`对象，包含`instruction`和`output`两个字段，其中`instruction`是输入的文本，`output`是对应的输出文本。

In [ ]:
!head -n 30 train_data/ch_poetry_train.json

[
    {
        "instruction": "写一首以“戌辰年孟冬月诗四首选一”为题的诗：",
        "output": "卜居白下傍烟霞，送别情怀乱似麻。何日游踪重北上，并门犹有旧时家。"
    },
    {
        "instruction": "写一首以“花甲述怀”为题的诗：",
        "output": "屈指悬弧日，匆匆六十年。诗书敦夙契，风月结前缘。心有千秋志，囊无一个钱。苍松撑傲骨，绝不受人怜。"
    },
    {
        "instruction": "写一首以“寓居夜感”为题的诗：",
        "output": "独坐晚凉侵，客窗秋意深。风沙疑化雾，夜气欲成霖。家务劳人倦，浓茶代酒斟。哀鸿鸣四野，一并助长吟。"
    },
    {
        "instruction": "写一首以“戏咏灵猫 其一”为题的诗：",
        "output": "灵猫雅号是金狮，水样柔情山样姿。早晚门前迎送我，撒娇投抱更依依。"
    },
    {
        "instruction": "写一首以“戏咏灵猫 其二”为题的诗：",
        "output": "金狮淘气喜翻书，对镜窥容恐不如。深愧家贫亏待了，眠无暖榻食无鱼。"
    },
    {
        "instruction": "写一首以“次答友人思乡诗”为题的诗：",
        "output": "阅尽沧桑万事空，何如归卧夕阳中。并州最是伤心地，四十馀年噩梦同。"
    },
    {
        "instruction": "写一首以“南归杂感 其一”为题的诗：",
        "output": "一到湘中耳目新，稻苗翻浪草铺茵。烟波无限江南趣，风柳非同塞北春。车辙移时山掠影，市声起处路扬尘。近乡莫怪行偏缓，屈指知交剩几人。"
    },
    {


在此，我们将使用默认的数据集，提交一个模型微调训练作业。SDK默认会打印训练作业在PAI控制台上的链接，用户可以通过该链接查看训练作业的状态和日志。

In [ ]:
est.fit(inputs=training_inputs)

`est.fit` 默认会打印训练作业日志，并等待到训练作业完成（成功，失败，或是被终止）。训练作业运行成功之后，用户可以通过查看`est.model_data()`查看训练作业的输入模型路径。

In [ ]:
print(est.model_data())